## About
The goal of this homework is to create a regression model for predicting housing prices (column 'median_house_value').
- Data is from kaggle (https://www.kaggle.com/datasets/camnugent/california-housing-prices)

#### Import Libraries

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import xgboost as xgb

In [22]:
pip install xgboost

  Using cached xgboost-2.0.1-py3-none-win_amd64.whl.metadata (2.0 kB)
   ---------------------------------------- 0.0/99.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.7 MB 330.3 kB/s eta 0:05:02
   ---------------------------------------- 0.0/99.7 MB 330.3 kB/s eta 0:05:02
   ---------------------------------------- 0.0/99.7 MB 330.3 kB/s eta 0:05:02
   ---------------------------------------- 0.0/99.7 MB 151.3 kB/s eta 0:10:59
   ---------------------------------------- 0.1/99.7 MB 218.8 kB/s eta 0:07:36
   ---------------------------------------- 0.1/99.7 MB 206.9 kB/s eta 0:08:02
   ---------------------------------------- 0.1/99.7 MB 249.8 kB/s eta 0:06:40
   ---------------------------------------- 0.1/99.7 MB 252.2 kB/s eta 0:06:36
   ---------------------------------------- 0.1/99.7 MB 252.2 kB/s eta 0:06:36
   ------

   ---------------------------- ----------- 71.3/99.7 MB 124.0 kB/s eta 0:03:50
   ---------------------------- ----------- 71.3/99.7 MB 124.0 kB/s eta 0:03:50
   ---------------------------- ----------- 71.3/99.7 MB 124.0 kB/s eta 0:03:50
   ---------------------------- ----------- 71.3/99.7 MB 124.0 kB/s eta 0:03:50
   ---------------------------- ----------- 71.3/99.7 MB 124.0 kB/s eta 0:03:50
   ---------------------------- ----------- 71.4/99.7 MB 122.2 kB/s eta 0:03:53
   ---------------------------- ----------- 71.4/99.7 MB 122.2 kB/s eta 0:03:53
   ---------------------------- ----------- 71.4/99.7 MB 122.2 kB/s eta 0:03:53
   ---------------------------- ----------- 71.4/99.7 MB 122.2 kB/s eta 0:03:53
   ---------------------------- ----------- 71.4/99.7 MB 122.2 kB/s eta 0:03:53
   ---------------------------- ----------- 71.4/99.7 MB 122.2 kB/s eta 0:03:53
   ---------------------------- ----------- 71.4/99.7 MB 122.2 kB/s eta 0:03:53
   ---------------------------- --------

#### Data Preparation
- First, keep only the records where ocean_proximity is either '<1H OCEAN' or 'INLAND'

- Preparation:

1. Fill missing values with zeros.
2. Apply the log transform to median_house_value.
3. Do train/validation/test split with 60%/20%/20% distribution.
4. Use the train_test_split function and set the random_state parameter to 1.
5. Use DictVectorizer(sparse=True) to turn the dataframes into matrices.

In [2]:
df = pd.read_csv('housing.csv')

df.shape

(20640, 10)

In [3]:
df.head().T

,0,1,2,3,4
longitude,-122.23,-122.22,-122.24,-122.25,-122.25
latitude,37.88,37.86,37.85,37.85,37.85
housing_median_age,41.0,21.0,52.0,52.0,52.0
total_rooms,880.0,7099.0,1467.0,1274.0,1627.0
total_bedrooms,129.0,1106.0,190.0,235.0,280.0
population,322.0,2401.0,496.0,558.0,565.0
households,126.0,1138.0,177.0,219.0,259.0
median_income,8.3252,8.3014,7.2574,5.6431,3.8462
median_house_value,452600.0,358500.0,352100.0,341300.0,342200.0
ocean_proximity,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY,NEAR BAY


In [4]:
#keep only the records where ocean_proximity is either '<1H OCEAN' or 'INLAND'

df = df[df['ocean_proximity'].isin(['<1H OCEAN', 'INLAND'])]

In [5]:
df.ocean_proximity.unique()

array(['<1H OCEAN', 'INLAND'], dtype=object)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15687 entries, 701 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           15687 non-null  float64
 1   latitude            15687 non-null  float64
 2   housing_median_age  15687 non-null  float64
 3   total_rooms         15687 non-null  float64
 4   total_bedrooms      15530 non-null  float64
 5   population          15687 non-null  float64
 6   households          15687 non-null  float64
 7   median_income       15687 non-null  float64
 8   median_house_value  15687 non-null  float64
 9   ocean_proximity     15687 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.3+ MB


In [7]:
df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        157
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [8]:
# Fill missing values with zeros
df.fillna(0, inplace=True)
df.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [9]:
# Apply a log transform to median_house_value
df['median_house_value'] = np.log1p(df['median_house_value'])

# Split the data into features (X) and target (y)
X = df.drop(columns=['median_house_value'])
y = df['median_house_value']

# Do train/validation/test split with 60%/20%/20% distribution.
# Use the train_test_split function and set the random_state parameter to 1.
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

# Use DictVectorizer to turn the dataframes into matrices
dv = DictVectorizer(sparse=True)

X_train = dv.fit_transform(X_train.to_dict(orient='records'))
X_val = dv.transform(X_val.to_dict(orient='records'))
X_test = dv.transform(X_test.to_dict(orient='records'))


## Question 1
- Let's train a decision tree regressor to predict the median_house_value variable.
- Train a model with max_depth=1.
- Which feature is used for splitting the data? ocean_proximity, total_rooms, latitude, population

In [10]:
model = DecisionTreeRegressor(max_depth=1)

model.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=1)

In [11]:
# Access the feature used for the split
feature_index = model.tree_.feature[0]

# Get the feature name
feature_name = dv.get_feature_names_out()[feature_index]

print("The feature used for splitting the data is:", feature_name)

The feature used for splitting the data is: ocean_proximity=<1H OCEAN


## Question 2
- Train a random forest model with these parameters: n_estimators=10, random_state=1, n_jobs=-1 (optional - to make training faster)
- What's the RMSE of this model on validation?

In [12]:
# Create a Random Forest Regressor with the specified parameters
rf_model = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)

In [13]:
# Train the model on the training data
rf_model.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [14]:
# Make predictions on the validation data
y_pred = rf_model.predict(X_val)
# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("RMSE on validation data:", rmse)

RMSE on validation data: 0.23494749481338612


## Question 3
- Now let's experiment with the n_estimators parameter

- Try different values of this parameter from 10 to 200 with step 10.
- Set random_state to 1.
- Evaluate the model on the validation dataset.
- After which value of n_estimators does RMSE stop improving? Consider 3 decimal places for retrieving the answer.

In [15]:
# Define a range of n_estimators values to experiment with
n_estimators_values = range(10, 201, 10)

# Initialize an empty list to store RMSE values
rmse_values = []

for n_estimators in n_estimators_values:
    # Create a Random Forest Regressor with the specified parameters
    rfr = RandomForestRegressor(n_estimators=n_estimators, random_state=1, n_jobs=-1)

    # Train the model on the training data
    rfr.fit(X_train, y_train)

    # Make predictions on the validation data
    y_val_pred = rfr.predict(X_val)

    # Calculate the RMSE
    rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

    rmse_values.append(rmse)
    print(f"n_estimators={n_estimators}: RMSE={rmse}")

# Find the index of the minimum RMSE value
best_n_estimators_index = np.argmin(rmse_values)

# Determine the best value of n_estimators
best_n_estimators = n_estimators_values[best_n_estimators_index]
print(f"The best value of n_estimators is {best_n_estimators} with RMSE={rmse_values[best_n_estimators_index]}")

n_estimators=10: RMSE=0.23494749481338617
n_estimators=20: RMSE=0.22604762790313948
n_estimators=30: RMSE=0.22328670466368325
n_estimators=40: RMSE=0.2231039901996149
n_estimators=50: RMSE=0.2216916148359469
n_estimators=60: RMSE=0.22163009601252193
n_estimators=70: RMSE=0.2211838247633606
n_estimators=80: RMSE=0.22105417144530698
n_estimators=90: RMSE=0.22139325914534086
n_estimators=100: RMSE=0.22089727183246022
n_estimators=110: RMSE=0.22046699461782873
n_estimators=120: RMSE=0.220512862698102
n_estimators=130: RMSE=0.2202565207102791
n_estimators=140: RMSE=0.22020131491943967
n_estimators=150: RMSE=0.22014408075868275
n_estimators=160: RMSE=0.22014655690222995
n_estimators=170: RMSE=0.2201709965215708
n_estimators=180: RMSE=0.22031187348544926
n_estimators=190: RMSE=0.22032865786624464
n_estimators=200: RMSE=0.22033044911610855
The best value of n_estimators is 150 with RMSE=0.22014408075868275


## Question 4
- Let's select the best max_depth:

1. Try different values of max_depth: [10, 15, 20, 25]
2. For each of these values,try different values of n_estimators from 10 till 200 (with step 10)
3. calculate the mean RMSE
4. Fix the random seed: random_state=1
    
- What's the best max_depth, using the mean RMSE?

In [16]:
# Define a range of max_depth and n_estimators values to experiment with
max_depth_values = [10, 15, 20, 25]
n_estimators_values = range(10, 201, 10)

# Initialize variables to track the best max_depth and corresponding RMSE
best_max_depth = None
best_rmse = float('inf')

for max_depth in max_depth_values:
    for n_estimators in n_estimators_values:
        # Create a Random Forest Regressor with the specified parameters
        rf_model = RandomForestRegressor(max_depth=max_depth, n_estimators=n_estimators, random_state=1, n_jobs=-1)

        # Train the model on the training data
        rf_model.fit(X_train, y_train)

        # Make predictions on the validation data
        y_val_pred = rf_model.predict(X_val)

        # Calculate the RMSE
        rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))

        # Check if this combination has a lower RMSE
        if rmse < best_rmse:
            best_max_depth = max_depth
            best_rmse = rmse

        print(f"max_depth={max_depth}, n_estimators={n_estimators}: RMSE={rmse}")

print(f"The best max_depth is {best_max_depth} with RMSE={best_rmse}")

max_depth=10, n_estimators=10: RMSE=0.24072991959893017
max_depth=10, n_estimators=20: RMSE=0.23685371056927054
max_depth=10, n_estimators=30: RMSE=0.23422965971218174
max_depth=10, n_estimators=40: RMSE=0.23373409003404927
max_depth=10, n_estimators=50: RMSE=0.23288388335165514
max_depth=10, n_estimators=60: RMSE=0.23314939685394795
max_depth=10, n_estimators=70: RMSE=0.2328841536757432
max_depth=10, n_estimators=80: RMSE=0.2322860873281373
max_depth=10, n_estimators=90: RMSE=0.232300622855486
max_depth=10, n_estimators=100: RMSE=0.2320837126294032
max_depth=10, n_estimators=110: RMSE=0.2317295067449522
max_depth=10, n_estimators=120: RMSE=0.23175016854341798
max_depth=10, n_estimators=130: RMSE=0.23161200650836491
max_depth=10, n_estimators=140: RMSE=0.23151075533976703
max_depth=10, n_estimators=150: RMSE=0.2315694801806739
max_depth=10, n_estimators=160: RMSE=0.23158160057078658
max_depth=10, n_estimators=170: RMSE=0.23154917115686482
max_depth=10, n_estimators=180: RMSE=0.23175831

## Question 5
- Train the model with these parameters: n_estimators=10, max_depth=20, random_state=1, n_jobs=-1 (optional)
- Get the feature importance information from this model
- What's the most important feature (among these 4)?

In [17]:
# Create model with the specified parameters
model = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)

# Train the model
model.fit(X_train, y_train)

# Get feature importances from the model
feature_importances = model.feature_importances_

# Create a dictionary to map feature names to their importance scores
feature_importance_dict = dict(zip(dv.get_feature_names_out(), feature_importances))

# Find the most important feature
most_important_feature = max(feature_importance_dict, key=feature_importance_dict.get)

print("The most important feature is:", most_important_feature)

The most important feature is: median_income


## Question 6
- Now let's train an XGBoost model! For this question, we'll tune the eta parameter:

1. Install XGBoost
2. Create DMatrix for train and validation
3. Create a watchlist
4. Train a model with these parameters for 100 rounds:
- xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,}
5. Now change eta from 0.3 to 0.1.

- Which eta leads to the best RMSE score on the validation dataset?

In [25]:
# Create DMatrix for train and validation
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

In [26]:
# Create a watchlist
watchlist = [(dval, 'eval'), (dtrain, 'train')]

# Define XGBoost parameters
xgb_params = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,
    'objective': 'reg:squarederror',
    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
    'eval_metric': 'rmse'  # RMSE as the evaluation metric
}


In [27]:
# Train the model with eta=0.3 for 100 rounds
num_round = 100
model_eta_0_3 = xgb.train(xgb_params, dtrain, num_round, watchlist)

# Make predictions on the validation data
y_pred_eta_0_3 = model_eta_0_3.predict(dval)

# Calculate RMSE for eta=0.3
rmse_eta_0_3 = np.sqrt(mean_squared_error(y_val, y_pred_eta_0_3))
print("RMSE with eta=0.3:", rmse_eta_0_3)


[0]	eval-rmse:0.44336	train-rmse:0.44258
[1]	eval-rmse:0.36612	train-rmse:0.36286
[2]	eval-rmse:0.31911	train-rmse:0.31398
[3]	eval-rmse:0.29298	train-rmse:0.28441
[4]	eval-rmse:0.27583	train-rmse:0.26506
[5]	eval-rmse:0.26474	train-rmse:0.25176
[6]	eval-rmse:0.25848	train-rmse:0.24301
[7]	eval-rmse:0.25011	train-rmse:0.23305
[8]	eval-rmse:0.24745	train-rmse:0.22820
[9]	eval-rmse:0.24424	train-rmse:0.22173
[10]	eval-rmse:0.24121	train-rmse:0.21631
[11]	eval-rmse:0.23734	train-rmse:0.21105
[12]	eval-rmse:0.23638	train-rmse:0.20836
[13]	eval-rmse:0.23565	train-rmse:0.20530
[14]	eval-rmse:0.23487	train-rmse:0.20205
[15]	eval-rmse:0.23304	train-rmse:0.19798
[16]	eval-rmse:0.23164	train-rmse:0.19560
[17]	eval-rmse:0.23090	train-rmse:0.19403


C:\Users\AMINAT\anaconda3\lib\site-packages\xgboost\core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[18]	eval-rmse:0.22898	train-rmse:0.19098
[19]	eval-rmse:0.22877	train-rmse:0.18807
[20]	eval-rmse:0.22786	train-rmse:0.18518
[21]	eval-rmse:0.22655	train-rmse:0.18301
[22]	eval-rmse:0.22575	train-rmse:0.18188
[23]	eval-rmse:0.22538	train-rmse:0.17963
[24]	eval-rmse:0.22425	train-rmse:0.17695
[25]	eval-rmse:0.22399	train-rmse:0.17484
[26]	eval-rmse:0.22397	train-rmse:0.17346
[27]	eval-rmse:0.22367	train-rmse:0.17165
[28]	eval-rmse:0.22247	train-rmse:0.16958
[29]	eval-rmse:0.22254	train-rmse:0.16847
[30]	eval-rmse:0.22247	train-rmse:0.16752
[31]	eval-rmse:0.22265	train-rmse:0.16602
[32]	eval-rmse:0.22191	train-rmse:0.16455
[33]	eval-rmse:0.22153	train-rmse:0.16360
[34]	eval-rmse:0.22129	train-rmse:0.16264
[35]	eval-rmse:0.22118	train-rmse:0.16119
[36]	eval-rmse:0.22124	train-rmse:0.15962
[37]	eval-rmse:0.22027	train-rmse:0.15760
[38]	eval-rmse:0.21992	train-rmse:0.15662
[39]	eval-rmse:0.21946	train-rmse:0.15556
[40]	eval-rmse:0.21964	train-rmse:0.15444
[41]	eval-rmse:0.21961	train-rmse:

In [28]:
# Now change eta to 0.1
xgb_params['eta'] = 0.1

# Train the model with eta=0.1 for 100 rounds
model_eta_0_1 = xgb.train(xgb_params, dtrain, num_round, watchlist)

# Make predictions on the validation data
y_pred_eta_0_1 = model_eta_0_1.predict(dval)

# Calculate RMSE for eta=0.1
rmse_eta_0_1 = np.sqrt(mean_squared_error(y_val, y_pred_eta_0_1))
print("RMSE with eta=0.1:", rmse_eta_0_1)


[0]	eval-rmse:0.52103	train-rmse:0.52259
[1]	eval-rmse:0.48526	train-rmse:0.48568
[2]	eval-rmse:0.45353	train-rmse:0.45291
[3]	eval-rmse:0.42568	train-rmse:0.42409
[4]	eval-rmse:0.40147	train-rmse:0.39883
[5]	eval-rmse:0.38047	train-rmse:0.37677
[6]	eval-rmse:0.36229	train-rmse:0.35762
[7]	eval-rmse:0.34683	train-rmse:0.34121
[8]	eval-rmse:0.33299	train-rmse:0.32671
[9]	eval-rmse:0.32094	train-rmse:0.31382
[10]	eval-rmse:0.31079	train-rmse:0.30297
[11]	eval-rmse:0.30148	train-rmse:0.29296
[12]	eval-rmse:0.29419	train-rmse:0.28471
[13]	eval-rmse:0.28772	train-rmse:0.27737
[14]	eval-rmse:0.28176	train-rmse:0.27037
[15]	eval-rmse:0.27667	train-rmse:0.26431
[16]	eval-rmse:0.27291	train-rmse:0.25953
[17]	eval-rmse:0.26896	train-rmse:0.25480


C:\Users\AMINAT\anaconda3\lib\site-packages\xgboost\core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[18]	eval-rmse:0.26600	train-rmse:0.25100
[19]	eval-rmse:0.26345	train-rmse:0.24753
[20]	eval-rmse:0.26035	train-rmse:0.24383
[21]	eval-rmse:0.25802	train-rmse:0.24085
[22]	eval-rmse:0.25551	train-rmse:0.23777
[23]	eval-rmse:0.25343	train-rmse:0.23509
[24]	eval-rmse:0.25134	train-rmse:0.23243
[25]	eval-rmse:0.24960	train-rmse:0.22987
[26]	eval-rmse:0.24817	train-rmse:0.22780
[27]	eval-rmse:0.24680	train-rmse:0.22574
[28]	eval-rmse:0.24541	train-rmse:0.22370
[29]	eval-rmse:0.24396	train-rmse:0.22139
[30]	eval-rmse:0.24255	train-rmse:0.21939
[31]	eval-rmse:0.24192	train-rmse:0.21809
[32]	eval-rmse:0.24068	train-rmse:0.21623
[33]	eval-rmse:0.23971	train-rmse:0.21432
[34]	eval-rmse:0.23880	train-rmse:0.21271
[35]	eval-rmse:0.23797	train-rmse:0.21113
[36]	eval-rmse:0.23747	train-rmse:0.20975
[37]	eval-rmse:0.23671	train-rmse:0.20828
[38]	eval-rmse:0.23607	train-rmse:0.20698
[39]	eval-rmse:0.23560	train-rmse:0.20585
[40]	eval-rmse:0.23461	train-rmse:0.20432
[41]	eval-rmse:0.23407	train-rmse:

In [29]:
# Compare RMSE scores and identify the best eta
if rmse_eta_0_3 < rmse_eta_0_1:
    best_eta = 0.3
elif rmse_eta_0_1 < rmse_eta_0_3:
    best_eta = 0.1
else:
    best_eta = "Both give equal value"

print("The best eta is:", best_eta)

The best eta is: 0.3
